In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import os
import pandas as pd
import xarray as xr
from matplotlib.projections.polar import PolarAxes
from scipy.interpolate import interp1d
from scipy.integrate import quad
from scipy.stats import linregress, weibull_min
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing as StandardScaler
from windrose import WindroseAxes
# Custom libraries
import data_loading
import plotting
import netcdf_exploration
import data_analysis

In [ ]:
# Base paths for different categories
measurements_path = 'data/measurements/'
turbine_info_path = 'data/turbine-info/coordinates/'
turbine_power_curves_path = 'data/turbine-info/power_curves/'
shapefiles_path = 'data/shapefiles/'
reanalysis_path = 'data/reanalysis/'

# Buyoy NetCDF files
bouy6_path = f'{measurements_path}2023-11-06_Buoy6_BSH_N-9.nc'
bouy2_path = f'{measurements_path}2023-11-09_Buoy2_BSH_N-9.nc'
# ERA5 D
era5_path = reanalysis_path
# Layouts base paths
turbines_existing_path = f'{turbine_info_path}existing/'
turbines_planned_future_path = f'{turbine_info_path}planned_future/'
turbines_planned_in_operation_before_2030_path = f'{turbine_info_path}planned_in_operation_before_2030/'
turbines_planned_netherlands_path = f'{turbine_info_path}planned_netherlands/'
turbines_area_of_interest_path = f'{turbine_info_path}area_of_interest/'
# Shapefiles paths
shapefiles_DEU_path = f'{shapefiles_path}DEU/*'
shapefiles_DNK_path = f'{shapefiles_path}DNK/*'
shapefiles_NLD_path = f'{shapefiles_path}NLD/*'

# Wind field layout files
file_N9_1 = f'{turbines_area_of_interest_path}layout-N-9.1.geom.csv'
file_N9_2 = f'{turbines_area_of_interest_path}layout-N-9.2.geom.csv'
file_N9_3 = f'{turbines_area_of_interest_path}layout-N-9.3.geom.csv'